In [1]:
import skopt
import numpy as np
import pandas as pd
import sliceoptim.samples as sp
import sliceoptim.core as core
import pickle
import os
from skopt.space import Real, Integer

In [2]:
printer = core.Printer(
    name="test_printer",
    bed_size=[220, 220],
    nozzle_diameter=0.4,
    max_speed=120,
    min_speed=5,
)
filament = core.Filament(
    name="pla_test",
    kind="pla",
    extrusion_temp_range=[180, 250],
    bed_temp_range=[25, 80],
    diameter=1.75,
)

In [3]:
### setup parameters space
params = []

In [4]:
params.append(Integer(name="first-layer-temperature", low=190, high=220))
params.append(Integer(name="first-layer-bed-temperature", low=40, high=55))
params.append(
    Integer(
        name="first-layer-speed", low=printer.min_speed, high=printer.max_speed * 0.5
    )
)
params.append(
    Real(
        name="first-layer-height",
        low=printer.nozzle_diameter * 0.7,#0.75,
        high=printer.nozzle_diameter,
    )
)
params.append(
    Real(
        name="first-layer-extrusion-width",
        low=printer.nozzle_diameter * 0.9,
        high=printer.nozzle_diameter * 2.5,
    )
)
params.append(Real(name="extrusion-multiplier", low=0.8, high=1.2))
params.append(
    Real(
        name="layer-height",
        low=printer.nozzle_diameter * 0.2,# 0.19,
        high=printer.nozzle_diameter,
    )
)

In [5]:
space = skopt.Space(params)

In [6]:
### setup experiment
fl_exp = core.Experiment(
    name="first_layer",
    sample_file="../assets/first_layer.stl",
    is_first_layer=True,
    spacing=5,
    printer=printer,
    filament=filament,
    params_space=space,
    output_file="../experiments/first_layer.gcode",
)

In [7]:
### lookup and save for existing data
data = pd.DataFrame()
for filename in os.listdir("../experiments/"):
    if filename.endswith(".csv"):
        # check if the file is related with expe and how
        if fl_exp.name in filename:
            data = data.append(pd.read_csv("../experiments/" + filename, index_col=0))

In [8]:
if True: ## TODO : samples grid id in csv export, estim_best_config min registered samples, unify from csv, correct rounding in csv
    fl_exp.create_new_sample_grid(15)
    fl_exp.import_csv("../experiments/first_layer.csv", overwrite_samples=True)
    fl_exp.compute_and_update_samples_costs()
    fl_exp.register_costs_to_optimizer()

In [9]:
data.tail()

,first-layer-temperature,first-layer-bed-temperature,first-layer-speed,first-layer-height,first-layer-extrusion-width,extrusion-multiplier,layer-height,temperature,bed-temperature,print_time,quality,cost,sample_grid_id
40,212,50,56,0.398406,0.377704,0.860451,0.357910,212,50,0.533772,7.5,NaN,2
41,204,52,31,0.385880,0.735323,0.976471,0.192342,204,52,0.590296,8.0,NaN,2
42,196,52,46,0.349145,0.508149,0.935149,0.303174,196,52,0.457728,9.0,NaN,2
43,210,54,11,0.334019,0.850594,1.101052,0.205064,210,54,1.017409,8.0,NaN,2
44,214,54,58,0.322368,0.717664,0.819923,0.173053,214,54,0.364084,8.5,NaN,2


### lookup for existing experiment object
for filename in os.listdir("../experiments/"):
    if filename == "first_layer.exp":
        with open("../experiments/first_layer.exp", "rb") as exp_file:
            fl_exp = pickle.load(exp_file)

## Generate next sample grid with csv for results (if not already done)

In [10]:
### first test
n_samples = 15
if data.empty:
    fl_exp.create_new_sample_grid(n_samples)
    fl_exp.write_gcode_for_last_sample_grid()
    fl_exp.to_csv("../experiments/first_layer.csv")
    with open("../experiments/first_layer.exp", "wb") as exp_file:
            pickle.dump(fl_exp, exp_file)
### other tests
else:
    if data["quality"].isna().any():
        print("You must fill all quality values in csv to proceed.")
    else:
        for i, s in enumerate(fl_exp.get_samples_list()):
            s.quality = data.loc[i, "quality"]
        fl_exp.compute_and_update_samples_costs()
        fl_exp.register_costs_to_optimizer()
        fl_exp.create_new_sample_grid(n_samples)
        fl_exp.write_gcode_for_last_sample_grid()
        fl_exp.export_csv("../experiments/first_layer.csv")
        with open("../experiments/first_layer.exp", "wb") as exp_file:
            pickle.dump(fl_exp, exp_file)


In [15]:
if fl_exp.get_samples_results_df()["cost"].isnull().any():
    fl_exp.sample_grid_list = fl_exp.sample_grid_list[:-1]
best_conf, std = fl_exp.estim_best_config()
print(best_conf)
print(std)

{'first-layer-temperature': 194.0, 'first-layer-bed-temperature': 47.0, 'first-layer-speed': 42.0, 'first-layer-height': 0.30380183449973774, 'first-layer-extrusion-width': 0.36187805920739535, 'extrusion-multiplier': 0.9608444769306863, 'layer-height': 0.08, 'temperature': 200.0, 'bed-temperature': 49.0}
[1.42270309]


In [16]:
fl_exp.write_validation_sample(best_conf, "../experiments/first_layer_validation.gcode")

### old results
({'first-layer-temperature': 200.0,
  'first-layer-bed-temperature': 41.0,
  'first-layer-speed': 56.0,
  'first-layer-height': 0.3883091239680703,
  'first-layer-extrusion-width': 0.7890043521390845,
  'extrusion-multiplier': 0.9461122801514417,
  'layer-height': 0.3138314145538781},
 array([1.18629722]))

 ({'first-layer-temperature': 207.0,
  'first-layer-bed-temperature': 54.0,
  'first-layer-speed': 60.0,
  'first-layer-height': 0.3640539917527952,
  'first-layer-extrusion-width': 0.6443718784490133,
  'extrusion-multiplier': 0.8,
  'layer-height': 0.16840278191077146},
 array([0.59983439]))